In [1]:
#Import Dependencies
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as bs
import time

In [2]:
#Launch Splinter
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

### Pulling Data from Race Results Page

In [3]:
#Visit Race Results Page 
url = "https://rankings.usatriathlon.org/Event/ViewEvent/301770/1"
browser.visit(url)

In [4]:
#we want the HTML from the page our browser is currently on
html = browser.html


#soupify (parse) the html so we can read it
soup = bs(html, 'html.parser')

In [5]:
#Find the data - in this case where the role = gridcell
athlete_data = soup.find_all(attrs={"role": "gridcell"})
athlete_data

[<div class="jqx-grid-cell jqx-grid-cell-energyblue" role="gridcell" style="left: 0px; z-index: 799; width:120px;" title="1"><div style="text-overflow: ellipsis; overflow: hidden; padding-bottom: 2px; text-align: center; margin-top: 5px;">1</div></div>,
 <div class="jqx-grid-cell jqx-grid-cell-energyblue" role="gridcell" style="left: 120px; z-index: 798; width:137px;" title="Michael"><div style="text-overflow: ellipsis; overflow: hidden; padding-bottom: 2px; text-align: center; margin-top: 5px;">Michael</div></div>,
 <div class="jqx-grid-cell jqx-grid-cell-energyblue" role="gridcell" style="left: 257px; z-index: 797; width:137px;" title="Alexander"><div style="text-overflow: ellipsis; overflow: hidden; padding-bottom: 2px; text-align: center; margin-top: 5px;">Alexander</div></div>,
 <div class="jqx-grid-cell jqx-grid-cell-energyblue" role="gridcell" style="left: 394px; z-index: 796; width:116px;" title="M"><div style="text-overflow: ellipsis; overflow: hidden; padding-bottom: 2px; tex

In [6]:
athlete_data[1]

<div class="jqx-grid-cell jqx-grid-cell-energyblue" role="gridcell" style="left: 120px; z-index: 798; width:137px;" title="Michael"><div style="text-overflow: ellipsis; overflow: hidden; padding-bottom: 2px; text-align: center; margin-top: 5px;">Michael</div></div>

In [7]:
#Extract the data and put it into a DataFrame 
count = 0
fullTable = []
currow = []

for cell in athlete_data:
    try:
            text = cell.find("div").text
            currow.append(text)
            count += 1
            if count == 9:
                fullTable.append(currow)
                currow = []
                count = 0
    except Exception as e: 
        print(e)
df = pd.DataFrame(fullTable)

In [8]:
df

,0,1,2,3,4,5,6,7,8
0,1,Michael,Alexander,M,53,FL,Age Group,01:02:29.000,87.912
1,2,Mark,Hulbert,M,54,FL,Age Group,01:04:06.000,85.695
2,3,Brian,Durden,M,35,FL,Age Group,01:05:30.000,83.863
3,4,Rodney,Carter,M,42,FL,Age Group,01:05:54.000,83.354
4,5,James,Hooppaw,M,27,FL,Age Group,01:06:10.000,83.018
5,6,Alexander,Williams,M,36,FL,Age Group,01:06:31.000,82.582
6,7,Benjamin,Peacock,M,23,FL,Age Group,01:07:12.000,81.742
7,8,Joel,Radd,M,33,FL,Age Group,01:08:02.000,80.741
8,9,KC,Wilson,M,26,FL,Age Group,01:09:14.000,79.341
9,10,Andrew,Martin,M,50,FL,Age Group,01:11:41.000,76.63


### Pulling URLS from Main Results Page

In [9]:
#Visit the main page 
url2 = "https://rankings.usatriathlon.org/Event/Events"
browser.visit(url2)

In [10]:
#Grab the HTML and soupify 
html2 = browser.html
soup2= bs(html2, 'html.parser')

In [11]:
#Bring back the URLS and race namesfor each race page 
links = soup2.find_all("a")
link_list = []
race_name = []
for anchor in links:
    links = anchor["href"]
    names = anchor.text
    
    link_list.append(links)
    race_name.append(names)

In [12]:
soup2.find_all("a")[4].text

'Bartow Blarney Triathlon'

In [13]:
#Create a DataFrame to hold the Race Name and the URLS - we'll iterate through this to extract data from each page
race_main = pd.DataFrame()
race_main['Race'] = race_name
race_main['link'] = ['https://rankings.usatriathlon.org'+ link for link in link_list]
#We'll be able to use try/except to filter out the pages that aren't actually race results

In [14]:
race_main

,Race,link
0,\n,https://rankings.usatriathlon.orghttp://usatri...
1,Race Results,https://rankings.usatriathlon.org/Event/Events
2,National Rankings,https://rankings.usatriathlon.org/Rankings/Nat...
3,Athlete Results,https://rankings.usatriathlon.org/RaceResult/A...
4,Bartow Blarney Triathlon,https://rankings.usatriathlon.org/Event/ViewEv...
5,College of the Florida Keys Sprint Triathlon/B...,https://rankings.usatriathlon.org/Event/ViewEv...
6,Florida Ultra Week,https://rankings.usatriathlon.org/Event/ViewEv...
7,UCSB Kendra's Race Triathlon,https://rankings.usatriathlon.org/Event/ViewEv...
8,Las Olas Fort Lauderdale Triathlon,https://rankings.usatriathlon.org/Event/ViewEv...
9,Pasadena Triathlon 2020,https://rankings.usatriathlon.org/Event/ViewEv...


### Put It All Together

In [15]:
# #check that each page is being visited/reference list
# for index,row in race_main.iterrows(): 
#     url3 = row.link
#     if '/ViewEvent' in url3:
#         race = row.Race
#         try:
#             browser.visit(url3)
#             html3 = browser.html
#             soup3 = bs(html3, 'html.parser')
#             print(race)
#         except Exception as e: 
#             print(e)

Bartow Blarney Triathlon
College of the Florida Keys Sprint Triathlon/Biathlon
Florida Ultra Week
UCSB Kendra's Race Triathlon
Las Olas Fort Lauderdale Triathlon
Pasadena Triathlon 2020
Clermont Draft Legal Challenge 
Tri WashU 
Stanford Treeathlon
3rd Annual Tri-Family Racing Lake Pleasant 
Desert Triathlon
Iceman Triathlon
Shelbyville Triathlon Series Race #1
Huntington's Disease Triathlon Sprint and Olympic - Racing 4 Research To Find a Cure
Chasing Caterpillars Triathlon 2020
Tritonman 2020
Ultraman Florida Triathlon
Winter Wonderland Triathlon
Tri-owa Indoor Triathlon
HITS Triathlon Series: Sarasota, FL


In [16]:
browser.quit()

In [49]:
# create an empty df
USAT_data = pd.DataFrame()

#for row in link_data...
for index,row in race_main.iterrows():
    #open a browswer
    browser = Browser('chrome', **executable_path, headless=True)
    #set the url as the link in the dataframe
    url3 = row.link
    #if the url is a race results page (as opposed to a FB link or a URL fragment) then...
    if '/ViewEvent' in url3:
        #set the race variable as the name of the Race
        race = row.Race
        try:
            #visit the browser, grab the html, and soupify it
            browser.visit(url3)
            
            #schleep - there was a mismatch between the time it took the site to load and the time it took to scrape 
            #so this will let splinter chill out for a sec and let us grab what we need
            time.sleep(3)
            
            #resume operations
            html3 = browser.html
            soup3 = bs(html3, 'html.parser')
            
            #find the athlete data
            triathlete_data = soup3.find_all(attrs={"role": "gridcell"})
            
            #get each row of data and store in df 
            count = 0
            fullTable = []
            currow = []
            for cell in triathlete_data:
                #insert another try/except pair for the Florida races, where the first div doesn't have any text
                try:
                    info = cell.find("div").text
                    currow.append(info)
                    count += 1
                    if count == 9:
                        fullTable.append(currow)
                        currow = []
                        count = 0
                except: 
                    next
            df = pd.DataFrame(fullTable)
            
            #add in a column with the race
            df['Race'] = race
            
            #add to the empty df
            USAT_data = USAT_data.append(df)
            
        #if it isn't successful, bring back the name of the race and the error
        except Exception as e: 
            print(race)
            print(e)
            
        #quit browser
        browser.quit()

In [50]:
USAT_data.Race.value_counts()

Iceman Triathlon                                                                        20
Pasadena Triathlon 2020                                                                 20
Winter Wonderland Triathlon                                                             20
UCSB Kendra's Race Triathlon                                                            20
Bartow Blarney Triathlon                                                                20
Tritonman 2020                                                                          20
3rd Annual Tri-Family Racing Lake Pleasant                                              20
Shelbyville Triathlon Series Race #1                                                    20
Desert Triathlon                                                                        20
Stanford Treeathlon                                                                     20
Clermont Draft Legal Challenge                                                          20

In [28]:
USAT_data

,Race,0,1,2,3,4,5,6,7,8
0,College of the Florida Keys Sprint Triathlon/B...,1,Benjamin,Hartell,M,34,FL,Age Group,00:52:59.000,83.921
1,College of the Florida Keys Sprint Triathlon/B...,2,John,Wile,M,34,FL,Age Group,00:54:46.000,81.189
2,College of the Florida Keys Sprint Triathlon/B...,3,Barton,Smith,M,41,FL,Age Group,00:56:26.000,78.791
3,College of the Florida Keys Sprint Triathlon/B...,4,Matthew,Weber,M,28,GA,Age Group,00:57:00.000,78.008
4,College of the Florida Keys Sprint Triathlon/B...,5,Ericka,Stokes,F,32,FL,Age Group,01:01:05.000,80.072
...,...,...,...,...,...,...,...,...,...,...
15,Tri-owa Indoor Triathlon,16,Tyler,Dennis,M,21,IA,Age Group,00:41:41.130,78.147
16,Tri-owa Indoor Triathlon,17,Jack,Stewart,M,20,IL,Age Group,00:43:33.700,74.781
17,Tri-owa Indoor Triathlon,18,Max,Wascher,M,19,IL,Age Group,00:43:38.000,74.658
18,Tri-owa Indoor Triathlon,19,Andrew,Nguyen,M,20,KS,Age Group,00:43:54.050,74.203


In [46]:
USAT_data = USAT_data.rename(columns = {USAT_data.columns[1]:'Rank', USAT_data.columns[2]:'First_Name',USAT_data.columns[3]:'Last_Name',USAT_data.columns[4]:'Sex',USAT_data.columns[5]:'Age',USAT_data.columns[6]:'State',
                            USAT_data.columns[7]:'Group',USAT_data.columns[8]:'Time',USAT_data.columns[9]:'Points'})

In [47]:
USAT_data

,Race,Rank,First_Name,Last_Name,Sex,Age,State,Group,Time,Points
0,College of the Florida Keys Sprint Triathlon/B...,1,Benjamin,Hartell,M,34,FL,Age Group,00:52:59.000,83.921
1,College of the Florida Keys Sprint Triathlon/B...,2,John,Wile,M,34,FL,Age Group,00:54:46.000,81.189
2,College of the Florida Keys Sprint Triathlon/B...,3,Barton,Smith,M,41,FL,Age Group,00:56:26.000,78.791
3,College of the Florida Keys Sprint Triathlon/B...,4,Matthew,Weber,M,28,GA,Age Group,00:57:00.000,78.008
4,College of the Florida Keys Sprint Triathlon/B...,5,Ericka,Stokes,F,32,FL,Age Group,01:01:05.000,80.072
...,...,...,...,...,...,...,...,...,...,...
15,Tri-owa Indoor Triathlon,16,Tyler,Dennis,M,21,IA,Age Group,00:41:41.130,78.147
16,Tri-owa Indoor Triathlon,17,Jack,Stewart,M,20,IL,Age Group,00:43:33.700,74.781
17,Tri-owa Indoor Triathlon,18,Max,Wascher,M,19,IL,Age Group,00:43:38.000,74.658
18,Tri-owa Indoor Triathlon,19,Andrew,Nguyen,M,20,KS,Age Group,00:43:54.050,74.203


In [48]:
USAT_data.to_csv('USAT_Data')